In [1]:
import json
import requests
import io
import base64
import numpy as np
from PIL import Image, PngImagePlugin


In [5]:
url_clip = "https://c1fec062ad76545851.gradio.live"
url_sd = "https://b7940f3a-82a6-4fad.gradio.live"

In [6]:
# Load initial image -- check file extension
img = "dot.png"
out_dir = "outputs2"

# SD settings
query = {
    "steps": 25,
    "sampler_index": "Euler", #Euler a
    "width": 512,
    "height": 512,
    "seed": -1,
    "cfg_scale": 5
    }

# Create folder
try:
    os.mkdir(os.path.join(out_dir))
except:
    print("directory already existing")

# Interrogate first image
with open(img, "rb") as image_file:
    img_string = "data:image/png;base64,"+str(base64.b64encode(image_file.read()).decode('utf-8'))

# Save first iter
filepath = out_dir+'/iter_0000.png'
image = Image.open(img)
image.save(filepath)


directory already existing


In [8]:
prompts = []

# Loop between sd and clip
for j in range(1,20):

    # Ask Clip #####################################################
    req_clip = {
        "data": [
            img_string,
            "fast",
            16,]
            }

    clip_response = requests.post(url=f'{url_clip}/run/predict', json=req_clip).json()
    print(clip_response)
    prompt = clip_response["data"][0]
    print(prompt)

    # Store prompt
    prompts.append(prompt)

    # Ask SD ########################################################
    # query["prompt"] = prompt.split(",")[0] # only subject
    query["prompt"] = prompt

    sd_response = requests.post(url=f'{url_sd}/sdapi/v1/txt2img', json=query).json()

    for i in sd_response['images']:
        image = Image.open(io.BytesIO(base64.b64decode(i.split(",",1)[0])))

        png_payload = {
            "image": "data:image/png;base64," + i
        }

        response2 = requests.post(url=f'{url_sd}/sdapi/v1/png-info', json=png_payload)
        pnginfo = PngImagePlugin.PngInfo()
        pnginfo.add_text("parameters", response2.json().get("info"))


        filepath = out_dir+'/iter_'+ '{:0>4}'.format(j) +'.png'
        image.save(filepath, pnginfo=pnginfo)

        # Override settings for next loop
        img_string = "data:image/png;base64," + i

# Store results of prompts
np.savetxt(out_dir+"/prompts.csv", 
           prompts,
           delimiter ="; ", 
           fmt ='% s')

{'error': 'This app has no endpoint /api/predict/.'}


KeyError: 'data'

In [33]:
# Store results of prompts in case of stop
np.savetxt(out_dir+"/prompts.csv", 
           prompts,
           delimiter ="; ", 
           fmt ='% s')

In [12]:
pip freeze

asttokens==2.2.1
backcall==0.2.0
-e git+https://github.com/pharmapsychotic/BLIP.git@5b58f59454039295a87c6628b1392e6b8752e911#egg=blip
certifi==2022.12.7
charset-normalizer==2.1.1
click==8.1.3
clip-interrogator==0.3.2
colorama==0.4.6
comm==0.1.2
contourpy==1.0.6
cycler==0.11.0
debugpy==1.6.4
decorator==5.1.1
entrypoints==0.4
executing==1.2.0
fairscale==0.4.4
filelock==3.8.2
fonttools==4.38.0
ftfy==6.1.1
huggingface-hub==0.11.1
idna==3.4
ipykernel==6.19.2
ipython==8.7.0
jedi==0.18.2
joblib==1.2.0
jupyter-client==7.4.8
jupyter-core==5.1.0
kiwisolver==1.4.4
matplotlib==3.6.2
matplotlib-inline==0.1.6
nest-asyncio==1.5.6
numpy==1.23.5
open-clip-torch==2.8.2
packaging==22.0
parso==0.8.3
pickleshare==0.7.5
Pillow==9.3.0
platformdirs==2.6.0
prompt-toolkit==3.0.36
protobuf==3.20.3
psutil==5.9.4
pure-eval==0.2.2
pycocoevalcap==1.2
pycocotools==2.0.6
Pygments==2.13.0
pyparsing==3.0.9
python-dateutil==2.8.2
pywin32==305
PyYAML==6.0
pyzmq==24.0.1
regex==2022.10.31
requests==2.28.1
sacremoses==0.0.53